In [1]:
import sympy as sp
from sympy import init_printing
init_printing()
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent))
from cores.dynamical_systems.inverted_pendulum import InvertedPendulum
import torch
import numpy as np

In [2]:
mass = 1.0
length = 1.0
viscous_friction = 0.1
gravity = 9.81

In [3]:
system_torch = InvertedPendulum(mass=mass, length=length, viscous_friction=viscous_friction, dtype=torch.float32)

In [4]:
theta, theta_dot = sp.symbols(r'\theta \dot{\theta}')
u = sp.symbols(r'u')
states = sp.Matrix([theta, theta_dot])
controls = sp.Matrix([u])

In [5]:
f1 = theta_dot
sin_theta = sp.sin(theta)

f2 = gravity / length * sin_theta + u / (mass * length ** 2) - viscous_friction / (mass * length ** 2) * theta_dot
f = sp.Matrix([f1, f2])

# Test f(x,u)

In [6]:
f_func = sp.lambdify((states, controls), f)
state_np = np.array([1., 2.], dtype=np.float32)
control_np = np.array([3.], dtype=np.float32)
print(f_func(state_np, control_np))


[[ 2.       ]
 [11.0548315]]


In [7]:
state_torch = torch.tensor(state_np, dtype=torch.float32).unsqueeze(0)
control_torch = torch.tensor(control_np, dtype=torch.float32).unsqueeze(0)
f_torch = system_torch(state_torch, control_torch)
print(f_torch)

tensor([[ 2.0000, 11.0548]])


# Test df/dx

In [10]:
df_dx = f.jacobian(states)
df_dx_func = sp.lambdify((states, controls), df_dx)
print(df_dx_func(state_np, control_np))

[[ 0.          1.        ]
 [ 5.30036545 -0.1       ]]


In [11]:
df_dx_torch = system_torch.f_dx(state_torch, control_torch)
print(df_dx_torch)

tensor([[[ 0.0000,  1.0000],
         [ 5.3004, -0.1000]]])


# Test df/du

In [12]:
df_du = f.jacobian(controls)
df_du_func = sp.lambdify((states, controls), df_du)
print(df_du_func(state_np, control_np))

[[0.]
 [1.]]


In [13]:
df_du_torch = system_torch.f_du(state_torch, control_torch)
print(df_du_torch)

tensor([[[0.],
         [1.]]])


# d^2f/dx^2

In [14]:
f1_dxdx = f1.diff(states).jacobian(states)
print(f1_dxdx)

Matrix([[0, 0], [0, 0]])


In [15]:
f2_dxdx = f2.diff(states).jacobian(states)
print(f2_dxdx)

Matrix([[-9.81*sin(\theta), 0], [0, 0]])
